In [22]:
from ema_workbench import (
    TimeSeriesOutcome, 
    ScalarOutcome,
    perform_experiments, 
    RealParameter,
    CategoricalParameter,
    ema_logging, 
    MultiprocessingEvaluator,
    ScalarOutcome,
    ArrayOutcome,
    Constant,
    Model,
    Policy
)
from ema_workbench.connectors.vensim import VensimModel
from ema_workbench.em_framework import CategoricalParameter
from ema_workbench.em_framework.salib_samplers import get_SALib_problem

In [23]:
import matplotlib.pyplot as plt
from ema_workbench.analysis.plotting import lines
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats.kde as kde


In [24]:
if __name__ == "__main__":
    ema_logging.log_to_stderr(ema_logging.INFO)
   

In [25]:
wd = r"./Models"
model = VensimModel("simpleModel", wd=wd, model_file="Thesismodel 4.vpmx")

In [26]:
model.outcomes = [
    TimeSeriesOutcome('Industrial CO2 savings with green hydrogen'),
    TimeSeriesOutcome("MW installed"),
    TimeSeriesOutcome("LCOH[green hydrogen]"),
    #TimeSeriesOutcome("satisfied demand"),
#    TimeSeriesOutcome("hydrogen type demand[green hydrogen]"),
#    TimeSeriesOutcome("kpi"),
    #TimeSeriesOutcome("total project funnel[green hydrogen]"),
    #TimeSeriesOutcome("Hydrogen type demand[green hydrogen]"),
    #TimeSeriesOutcome("total development time[green hydrogen]"),
 #   ScalarOutcome('max mw installed', variable_name='MW installed',function=np.max),
#    ScalarOutcome('min production costs', variable_name='LCOH[green hydrogen]',function=np.min),

]

In [27]:
model.uncertainties = [
#Green H2
    RealParameter("Learning rate",0.04,0.06),
    RealParameter("Innovation cycle",4,6),
    RealParameter("Relative elasticity",-0.8,-0.4),
    RealParameter("Market adaptation speed",0.4,0.8),
    RealParameter("Scaling factor",0.5,2),
    RealParameter("Electrolysis development",0.08,0.31),
    RealParameter("Max efficiency",0.8,0.9),
    RealParameter("CO2 elasticity", 0.4,0.8),
    RealParameter("Market maturity",0.2,0.6),
    RealParameter("Initial CAPEX", 4000, 6500),
    RealParameter("Operation & maintenance costs",0.05,0.15),
#H2 Demand
    RealParameter("Demand substitution rate",0.1,2),
    RealParameter("Price elasticity of demand[Green hydrogen]",-3,-1.5),
    RealParameter("Price elasticity of demand[Blue hydrogen]",-2.5,-1,),
    RealParameter("Price elasticity of demand[Grey hydrogen]",-1,0),
    RealParameter("Substitution delay",1.5,4.5),
    RealParameter("Premium value[Refineries]",0,2000),
    RealParameter("Premium value[Methanol]",0,2000),
    RealParameter("Premium value[Ammonia]",0,2000),
    RealParameter("Premium value[Steel]",0,2000),
    RealParameter("Premium value[Transport]",0,2000),
    RealParameter("Premium value[Synthetic fuels]",0,2000),
    RealParameter("Allocation key SDE",0,0.2),
    RealParameter("Allocation key",0,0.2),
    
# #RES

    RealParameter("Roadmap WOZ",1000,2000),
    RealParameter("Onshore RES increase",0.01,0.04),
    RealParameter("Other additional renewables",0.01,0.04),
    RealParameter("Domestic electrification",0.03,0.05),
    RealParameter("Min Capex",500,2000),
    RealParameter("Policy market size adjustment[Refineries]",0.01,0.03),
    RealParameter("Policy market size adjustment[Methanol]",-0.02,0.02),
    RealParameter("Policy market size adjustment[Transport]",-0.02,0.02),
    RealParameter("Policy market size adjustment[Ammonia]",-0.02,0.02),
    RealParameter("Policy market size adjustment[EFuels]",-0.02,0.02),
    
# #Energy demand
    RealParameter("Transport fuel substitution rate", 0.1,0.2),
    RealParameter("Industrial energy substitution rate",0.1,0.2),
    RealParameter("Long term price effect",0.2,0.4),
    RealParameter("Short term price effect",0.1,0.3),
    RealParameter("Oil price",70,110),
    RealParameter("Coal price",70,110),
    RealParameter("Initial gas price",50,70),
    RealParameter("Initial electricity price",50,70),
    RealParameter("Biofuel price",1900,2100),
    RealParameter("Efuel price",2400,2600),
    RealParameter("Market reaction delay",0.8,1.2),
    RealParameter("Energy carrier demand elasticity",-0.6,-0.4),
    RealParameter("Business cycle",5,7),
    RealParameter("Average GDP growth",0.01,0.03),
    RealParameter("Innovation factor[Synthetic fuels]",0.01,0.02),
    RealParameter("Innovation factor[Biofuels]",0.01,0.02),
    RealParameter("Innovation factor[Electricity]",0.01,0.02),
    RealParameter("Efficiency increase",0.01,0.02)
                  ]

In [28]:
policies = [
        Policy("No policy", model_file=r"Thesismodel no policy.vpmx"),
        Policy("Existing policy", model_file=r"Thesismodel exisiting policy.vpmx"),
        Policy("New policy", model_file=r"Thesismodel new policy.vpmx"),
    ]

In [29]:
from ema_workbench import MultiprocessingEvaluator, ema_logging, perform_experiments

ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(model, n_processes=7) as evaluator:
    results = evaluator.perform_experiments(scenarios=1000, policies = policies)

C:\Users\bern\anaconda3\lib\site-packages\ema_workbench\em_framework\evaluators.py:349: UserWarning: the number of processes cannot be more then 8
  warnings.warn(f"the number of processes cannot be more then {max_processes}")
[MainProcess/INFO] pool started with 7 workers
[MainProcess/INFO] performing 1000 scenarios * 3 policies * 1 model(s) = 3000 experiments
100%|██████████████████████████████████████| 3000/3000 [10:06<00:00,  4.95it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [30]:
experiments, outcomes = results

In [31]:
from ema_workbench import save_results
save_results(results, r'./1104.tar.gz')

C:\Users\bern\anaconda3\lib\site-packages\ema_workbench\em_framework\outcomes.py:496: UserWarning: still to be tested!!
  warnings.warn("still to be tested!!")
[MainProcess/INFO] results saved successfully to C:\Users\bern\Documents\TU\EPA\Thesis\Python\1104.tar.gz
